# Middle

In [ ]:
import cv2
import numpy as np
import os

# Input folder path and output folder path
input_folder = './hinhanh_19-21-122024/full/'
output_folder = './middle_full/'

# Check if the output folder does not exist, then create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List of image files in the input folder
image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Loop through each image in the input folder
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    
    # Read and process the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Cannot read the image {image_file}, skipping this file.")
        continue
    
    image = cv2.resize(image, (300, 400))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur and Canny edge detection
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get image size and center
    image_height, image_width = image.shape[:2]
    image_center = (image_width // 2, image_height // 2)

    # Minimum rectangle size threshold
    min_width = 30
    min_height = 30

    # Variables to store the closest and smallest rectangle
    closest_rect = None
    closest_distance = float('inf')
    min_area = float('inf')

    # Loop through each contour to find the closest and smallest rectangle
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        # Check if the rectangle size is above the threshold
        if w >= min_width and h >= min_height:
            rect_center = (x + w // 2, y + h // 2)
            distance_from_center = np.sqrt((rect_center[0] - image_center[0]) ** 2 + (rect_center[1] - image_center[1]) ** 2)
            area = w * h

            if distance_from_center < closest_distance or (distance_from_center == closest_distance and area < min_area):
                closest_rect = (x, y, w, h)
                closest_distance = distance_from_center
                min_area = area

    # If a suitable rectangle is found, apply the next step with K-means clustering
    if closest_rect is not None:
        x, y, w, h = closest_rect
        side = min(w, h)

        # Create a mask for the small square
        small_square_mask = np.zeros_like(gray_image)
        cv2.rectangle(small_square_mask, (x, y), (x + side, y + side), 255, -1)

        # Extract the region of interest (ROI) inside the small square
        small_square_roi = cv2.bitwise_and(image, image, mask=small_square_mask)
        roi_pixels = small_square_roi[np.where(small_square_mask == 255)]

        # Reshape ROI for K-means (each pixel is a data point)
        roi_reshaped = roi_pixels.reshape((-1, 3)).astype(np.float32)

        # Apply K-means clustering with k=2 to the small square
        k = 2
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        _, labels, centers = cv2.kmeans(roi_reshaped, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

        # Get the unique labels and their counts for the small square
        unique, counts = np.unique(labels, return_counts=True)
        min_cluster_percentage = (min(counts) / counts.sum()) * 100

        # Check if any cluster in the small square has less than 30% of the pixels
        if min_cluster_percentage < 30:
            draw_large_square = True

            # Calculate the dimensions for the larger square (3x the area)
            larger_side = int(np.sqrt(3) * side)
            larger_x = x - (larger_side - side) // 2
            larger_y = y - (larger_side - side) // 2

            # Ensure the larger square is within image bounds
            if (larger_x >= 0 and larger_y >= 0 and
                larger_x + larger_side <= image.shape[1] and
                larger_y + larger_side <= image.shape[0]):

                # Create masks for the large square and the ring area
                large_square_mask = np.zeros_like(gray_image)
                cv2.rectangle(large_square_mask, (larger_x, larger_y), (larger_x + larger_side, larger_y + larger_side), 255, -1)
                ring_mask = cv2.bitwise_xor(large_square_mask, small_square_mask)

                # Extract the region of interest (ROI) inside the ring mask
                ring_roi = cv2.bitwise_and(image, image, mask=ring_mask)
                roi_pixels = ring_roi[np.where(ring_mask == 255)]

                # Reshape ROI for K-means (each pixel is a data point)
                roi_reshaped = roi_pixels.reshape((-1, 3)).astype(np.float32)

                # Apply K-means clustering with k=2 to the ring area
                _, labels, centers = cv2.kmeans(roi_reshaped, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

                # Find the dominant cluster (the one with the most pixels)
                unique, counts = np.unique(labels, return_counts=True)
                dominant_cluster_index = unique[np.argmax(counts)]

                # Create a mask for the largest cluster in the ring area
                cluster_mask = np.zeros_like(ring_mask)
                ring_positions = np.where(ring_mask == 255)
                labels_reshaped = labels.flatten()
                cluster_mask[ring_positions] = np.where(labels_reshaped == dominant_cluster_index, 255, 0).reshape(-1)

                # Find contours of the dominant cluster mask
                contours, _ = cv2.findContours(cluster_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                # Find the largest contour
                if contours:
                    largest_contour = max(contours, key=cv2.contourArea)

                    # Get the bounding box of the largest contour
                    x, y, w, h = cv2.boundingRect(largest_contour)

                    # Calculate the area of the cropped region
                    area = w * h
                    new_side = int(np.sqrt(area / 3))

                    # Calculate the coordinates of the new square to keep its center aligned
                    center_x = x + w // 2
                    center_y = y + h // 2
                    new_x = center_x - new_side // 2
                    new_y = center_y - new_side // 2

                    # Ensure the new square is within image boundaries
                    new_x = max(0, new_x)
                    new_y = max(0, new_y)
                    new_x_end = min(image.shape[1], new_x + new_side)
                    new_y_end = min(image.shape[0], new_y + new_side)

                    # Crop the new small square from the original image
                    new_crop = image[new_y:new_y_end, new_x:new_x_end]

                    # Resize the new small square to 256x256
                    resized_crop = cv2.resize(new_crop, (256, 256))

                    # Save the result to the output folder
                    output_image_path = os.path.join(output_folder, os.path.basename(image_file))
                    cv2.imwrite(output_image_path, resized_crop)
                    print(f"Processed with large square (1/3 area): {output_image_path}")

print("Image processing has been completed.")


# Cover

In [ ]:
import cv2
import numpy as np
import os

# Input folder path and output folder path
input_folder = './hinhanh_19-21-122024/full/'
output_folder = './cover_full/'

# Check if the output folder does not exist, then create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List of image files in the input folder
image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

# Loop through each image in the input folder
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    
    # Read and process the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Cannot read the image {image_file}, skipping this file.")
        continue
    
    image = cv2.resize(image, (300, 400))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur and Canny edge detection
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    edges = cv2.Canny(blurred_image, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get image size and center
    image_height, image_width = image.shape[:2]
    image_center = (image_width // 2, image_height // 2)

    # Minimum rectangle size threshold
    min_width = 30
    min_height = 30

    # Variables to store the closest and smallest rectangle
    closest_rect = None
    closest_distance = float('inf')
    min_area = float('inf')

    # Loop through each contour to find the closest and smallest rectangle
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        # Check if the rectangle size is above the threshold
        if w >= min_width and h >= min_height:
            rect_center = (x + w // 2, y + h // 2)
            distance_from_center = np.sqrt((rect_center[0] - image_center[0]) ** 2 + (rect_center[1] - image_center[1]) ** 2)
            area = w * h

            if distance_from_center < closest_distance or (distance_from_center == closest_distance and area < min_area):
                closest_rect = (x, y, w, h)
                closest_distance = distance_from_center
                min_area = area

    # If a suitable rectangle is found, apply the next step with K-means clustering
    if closest_rect is not None:
        x, y, w, h = closest_rect
        side = min(w, h)

        # Create a mask for the small square
        small_square_mask = np.zeros_like(gray_image)
        cv2.rectangle(small_square_mask, (x, y), (x + side, y + side), 255, -1)

        # Extract the region of interest (ROI) inside the small square
        small_square_roi = cv2.bitwise_and(image, image, mask=small_square_mask)
        roi_pixels = small_square_roi[np.where(small_square_mask == 255)]

        # Reshape ROI for K-means (each pixel is a data point)
        roi_reshaped = roi_pixels.reshape((-1, 3)).astype(np.float32)

        # Apply K-means clustering with k=2 to the small square
        k = 2
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
        _, labels, centers = cv2.kmeans(roi_reshaped, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

        # Get the unique labels and their counts for the small square
        unique, counts = np.unique(labels, return_counts=True)
        min_cluster_percentage = (min(counts) / counts.sum()) * 100

        # Check if any cluster in the small square has less than 30% of the pixels
        if min_cluster_percentage < 30:
            draw_large_square = True

            # Calculate the dimensions for the larger square (3x the area)
            larger_side = int(np.sqrt(3) * side)
            larger_x = x - (larger_side - side) // 2
            larger_y = y - (larger_side - side) // 2

            # Ensure the larger square is within image bounds
            if (larger_x >= 0 and larger_y >= 0 and
                larger_x + larger_side <= image.shape[1] and
                larger_y + larger_side <= image.shape[0]):

                # Create masks for the large square and the ring area
                large_square_mask = np.zeros_like(gray_image)
                cv2.rectangle(large_square_mask, (larger_x, larger_y), (larger_x + larger_side, larger_y + larger_side), 255, -1)
                ring_mask = cv2.bitwise_xor(large_square_mask, small_square_mask)

                # Extract the region of interest (ROI) inside the ring mask
                ring_roi = cv2.bitwise_and(image, image, mask=ring_mask)
                roi_pixels = ring_roi[np.where(ring_mask == 255)]

                # Reshape ROI for K-means (each pixel is a data point)
                roi_reshaped = roi_pixels.reshape((-1, 3)).astype(np.float32)

                # Apply K-means clustering with k=2 to the ring area
                _, labels, centers = cv2.kmeans(roi_reshaped, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

                # Find the dominant cluster (the one with the most pixels)
                unique, counts = np.unique(labels, return_counts=True)
                dominant_cluster_index = unique[np.argmax(counts)]

                # Create a mask for the largest cluster in the ring area
                cluster_mask = np.zeros_like(ring_mask)
                ring_positions = np.where(ring_mask == 255)
                labels_reshaped = labels.flatten()
                cluster_mask[ring_positions] = np.where(labels_reshaped == dominant_cluster_index, 255, 0).reshape(-1)

                # Find contours of the dominant cluster mask
                contours, _ = cv2.findContours(cluster_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                # Find the largest contour
                if contours:
                    largest_contour = max(contours, key=cv2.contourArea)

                    # Get the bounding box of the largest contour
                    x, y, w, h = cv2.boundingRect(largest_contour)

                    # Crop the region from the original image
                    cropped_region = image[y:y + h, x:x + w]

                    # Resize to 256x256
                    resized_crop = cv2.resize(cropped_region, (256, 256))

                    # Save the result to the output folder
                    output_image_path = os.path.join(output_folder, os.path.basename(image_file))
                    cv2.imwrite(output_image_path, resized_crop)
                    print(f"Processed with large square: {output_image_path}")

                else:
                    print(f"No contour found in the dominant cluster for {image_file}")

        else:
            # If the large square cannot be created, crop the small square and resize
            small_crop = image[y:y + side, x:x + side]
            resized_crop = cv2.resize(small_crop, (256, 256))

            # Save the result to the output folder
            output_image_path = os.path.join(output_folder, os.path.basename(image_file))
            cv2.imwrite(output_image_path, resized_crop)
            print(f"Processed with small square: {output_image_path}")

print("Image processing has been completed.")